# Asymmetry Case Break Downs

In [1]:
from py2neo import Node, Relationship, Graph
from numpy.random import rand
import numpy as num
from bokeh.plotting import output_notebook,figure, show
from bokeh.layouts import row,column,gridplot
from bokeh.models import Label
import numpy as np
from curve import *
from bokeh.models import Span,Legend
output_notebook()

Loading BokehJS ...

In [2]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [3]:
import pickle as pk
A = pk.load(open('local.pk','rb'))


import random

def gen_hex_colour_code():
   return '#'+''.join([random.choice('0123456789ABCDEF') for x in range(6)])

C = {}

In [4]:
l = len(A)
A_dic = {}
for i in range(l):
    tmp = A[i]
    n1 = tmp['n1']
    n2 = tmp['n2']
    A_dic[n1+n2] = i

In [5]:
def roi_rot(w):
    p = w.split('L')
    return 'L'+p[-1]+'L'+p[1]
def logit(t):
    tmp = max(1,t)
    return num.log(tmp/5000.0)

In [6]:
null = {}
for i in range(1,181):
    for j in range(1,181):
        if (i==j):continue
        ind = A_dic['L'+str(i)+'L'+str(j)]
        tmp = A[ind]['weight']
        tmp = logit(tmp)
        if (tmp < -4.605170185988091):
            null['L'+str(i)+'L'+str(j)] = 1
        else:
            null['L'+str(i)+'L'+str(j)] = 0

In [30]:
total = sum(null.values()) # total null connections
asym = 0 # total asymmetric null connections
raw = [] #asymmtric case max weights in both directions
for w in null.keys():
    if null[w] == 1 and null[roi_rot(w)] == 0:
        asym = asym+1
        ind1 = A_dic[w]
        t1 = A[ind1]['weight']
        s1 = logit(t1)
        ind2 = A_dic[roi_rot(w)]
        t2 = A[ind2]['weight']
        s2 = logit(t2)
        raw.append((s1,s2))

# Case 1: Below the Noise floor in at least one direction
We call a connection below the noise floor a `null` connection. If a null conncetion A->B is not null on opposit direction B->A, it's `asymmetric null` connection. For 180 ROIs there are a total of __32220__ possible connections. Below we look at what percentage of these connections are `null` and `asymmetric null`. (NULL Threshold at __-4.6__)

In [31]:
print('Total null connections:'+str(total))
print('Total null connections (percentage of all connections):'+str(total*100/32220))
print('Total asymmetric null connections:'+str(asym))
print('Total asymmetric null connections (percentage of all null connections):'+str(asym*100/total))

Total null connections:26405
Total null connections (percentage of all connections):81.9522036002483
Total asymmetric null connections:1745
Total asymmetric null connections (percentage of all null connections):6.608596856655937


### Distribution of null connections strength asymmetry
For the `aymmetric null` connections let's look at the distribution of strength for the __NOT__ null direction.

In [10]:
mp = list(map(lambda x:x[1],raw))
p1 = figure(plot_width=400, plot_height=400,title = 'Asummetry in null cases',y_axis_label = "Distribution",x_axis_label = "Opposite direction")
q1,q2 = np.percentile(mp,[0,100])

measured = [xx for xx in mp if (xx>q1 and xx<q2)]
hist, edges = np.histogram(measured, density=True, bins=50)

p1.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],fill_color="#036564", line_color="#033649")

mp = list(map(lambda x:abs(x[1]-x[0]),raw))
p2 = figure(plot_width=400, plot_height=400,title = 'Asummetry in null cases',y_axis_label = "Distribution",x_axis_label = "Difference (abs)")
q1,q2 = np.percentile(mp,[0,100])

measured = [xx for xx in mp if (xx>q1 and xx<q2)]
hist, edges = np.histogram(measured, density=True, bins=50)

p2.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],fill_color="#036564", line_color="#033649")
grid = gridplot([[p1, p2]])
show (grid)


# CASE 2: 0-distance Connection
A connection that has __none-zero__ strength at __distance < .5__ is called 0-distance. Most of these connection are coming from adjacent ROIs. We look at 0-distance ROIs, their adjacency and their asymmetry.

In [15]:
# NEO4J database call for GROUP 1 - Close and Connected
graph = Graph(password="1234")
c1 = '''match (n:ROI)-[r:NOS]->(m:ROI)
where n.hemisphere="L" and m.hemisphere="L" and r.weight>.5
return r._length as dis,r._weight as wei,n.name as n1,m.name as n2, r.border as bor'''

D0 = graph.run(c1)

In [17]:
dis0 = {}
for con in D0:
    z = zip(con['dis'],con['wei'])
    tf = list(filter(lambda x: .1<x[0]<1 and x[1]>.5,z))
    if len(tf) == 0:
        continue
    else:
        dis0[con['n1']+con['n2']] = (max([xx[1] for xx in tf]),con['bor'])

In [32]:
total = len(dis0) # total adjacent connections
bor = 0
asym = 0 # total asymmetric adjacent connections
raw = [] #asymmtric case max weights in both directions
for w in dis0.keys():
    s1 = logit(dis0[w][0])
    if dis0[w][1]>0:
        bor = bor + 1
    try:
        s2 = logit(dis0[roi_rot(w)][0])
    except:
        asym = asym + 1
        s2 = logit(0)
    raw.append((s1,s2))

In [33]:
print('Total 0-distance connections:'+str(total))
print('Total 0-distance connections (percentage of all connections):'+str(total*100/32220))
print('Percentage of adjacency in 0-distance connection:'+ str(bor*100/total))

Total 0-distance connections:1049
Total 0-distance connections (percentage of all connections):3.255741775294848
Percentage of adjacency in 0-distance connection:97.52144899904671


### Case 2 strength asymmetry distribution

In [24]:
mp = list(map(lambda x:abs(x[1]-x[0]),raw))
p1 = figure(plot_width=400, plot_height=400,title = 'Asummetry in 0-distance cases',y_axis_label = "Distribution",x_axis_label = "Difference in connection strength between directions (abs)")
q1,q2 = np.percentile(mp,[0,100])

measured = [xx for xx in mp if (xx>q1 and xx<q2)]
hist, edges = np.histogram(measured, density=True, bins=50)

p1.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],fill_color="#036564", line_color="#033649")
show(p1)

# CASE 3: Good Local Regressors

In [25]:
total = 0
sym = 0
raw = []
for w in null:
    ind1 = A_dic[w]
    ind2 = A_dic[roi_rot(w)]
    if (A[ind1]['isSuccess']):
        total = total + 1
        if (A[ind2]['isSuccess']):
            sym = sym + 1
            s1 = logit(A[ind1]['weight'])
            s2 = logit(A[ind2]['weight'])
            raw.append((s1,s2))

In [26]:
print('Total good-local regressor connections:'+str(total))
print('Total good-local regressor connections (percentage of all connections):'+str(total/32220))
print('Total symmetric good-local regressor connections:'+str(sym))
print('Total symmetric good-local regressor connections (percentage of all null connections):'+str(sym/total))

Total good-local regressor connections:5081
Total good-local regressor connections (percentage of all connections):0.15769708255741774
Total symmetric good-local regressor connections:2944
Total symmetric good-local regressor connections (percentage of all null connections):0.5794135012792757


In [200]:
mp = list(map(lambda x:abs(x[1]-x[0]),raw))
p1 = figure(plot_width=400, plot_height=400,title = 'Asummetry in 0-distance cases',y_axis_label = "Distribution",x_axis_label = "Difference in connection strength between directions (abs)")
q1,q2 = np.percentile(mp,[0,100])

measured = [xx for xx in mp if (xx>q1 and xx<q2)]
hist, edges = np.histogram(measured, density=True, bins=50)

p1.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],fill_color="#036564", line_color="#033649")
show(p1)

# CASE 4: BAD LOCAL REGRESSOR

In [27]:
total = 0
sym = 0
raw = []
for w in null:
    ind1 = A_dic[w]
    ind2 = A_dic[roi_rot(w)]
    if (not A[ind1]['isSuccess']):
        total = total + 1
        if (not A[ind2]['isSuccess']):
            sym = sym + 1
            s1 = logit(A[ind1]['weight'])
            s2 = logit(A[ind2]['weight'])
            raw.append((s1,s2))

In [28]:
print('Total bad-local regressor connections:'+str(total))
print('Total bad-local regressor connections (percentage of all connections):'+str(total/32220))
print('Total symmetric bad-local regressor connections:'+str(sym))
print('Total symmetric bad-local regressor connections (percentage of all null connections):'+str(sym/total))

Total bad-local regressor connections:27139
Total bad-local regressor connections (percentage of all connections):0.8423029174425822
Total symmetric bad-local regressor connections:25002
Total symmetric bad-local regressor connections (percentage of all null connections):0.9212572312907624


In [29]:
mp = list(map(lambda x:abs(x[1]-x[0]),raw))
p1 = figure(plot_width=400, plot_height=400,title = 'Asymmetry in bad local regressor cases',y_axis_label = "Distribution",x_axis_label = "Difference in connection strength between directions (abs)")
q1,q2 = np.percentile(mp,[0,100])

measured = [xx for xx in mp if (xx>q1 and xx<q2)]
hist, edges = np.histogram(measured, density=True, bins=50)

p1.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],fill_color="#036564", line_color="#033649")
show(p1)